# Question 1

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# name of target column in our dataset
TARGET = "Annual_Income"
df = pd.read_csv('A2_Q1.csv')
df.head()

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
0,1,39,bachelors,never married,professional,high
1,2,50,doctorate,married,professional,mid
2,3,18,high school,never married,agriculture,low
3,4,30,bachelors,married,professional,mid
4,5,37,high school,married,agriculture,mid


## Part A
Compute impurity of target feature "Annual_Income"

For code resusability, we create a function to calculate and return impurity of a feature.<br>
Accepted impurity criterions:
- 'entropy' : Shannon's entropy
- 'gini' : Gini Index

In [2]:
# Referenced from https://www.featureranking.com/tutorials/machine-learning-tutorials/information-gain-computation/
def compute_impurity(feature, impurity_criterion):
    """
    This function calculates impurity of a feature.
    Supported impurity criteria: 'entropy', 'gini'
    input: feature (this needs to be a Pandas series)
    output: feature impurity
    """
    probs = feature.value_counts(normalize=True)
    
    if impurity_criterion == 'entropy':
        impurity = -1 * np.sum(np.log2(probs) * probs)
    elif impurity_criterion == 'gini':
        impurity = 1 - np.sum(np.square(probs))
    else:
        raise ValueError('Unknown impurity criterion')
        
    return(round(impurity, 3))

**We calculate impurity for target feature 'Annual_Income':**

- Using Gini Index:

In [3]:
compute_impurity(df[TARGET], 'gini')

0.555

- Using Shannon's entropy:

In [4]:
compute_impurity(df[TARGET], 'entropy')

1.353

## Part B
In this part we have to identify root node for our Decision Tree.<br>
We calculate **Information Gain** for different splits on all features and choose the split which gives the most **Information Gain**.

**Now we handle the continuos "Age" feature **<br>
We first sort the input df in ascending order by Age

In [5]:
df = df.sort_values(by=['Age'])
df.head()

,ID,Age,Education,Marital_Status,Occupation,Annual_Income
2,3,18,high school,never married,agriculture,low
5,6,23,high school,never married,agriculture,low
12,13,23,bachelors,never married,agriculture,low
19,20,25,bachelors,married,transport,high
13,14,25,high school,married,professional,high


As we can see above, our df is sorted by Age.

We now define a function to calculate Information Gain for all features of our dataframe,

In [6]:
# Referenced from https://www.featureranking.com/tutorials/machine-learning-tutorials/information-gain-computation/
def comp_feature_information_gain(df, target, descriptive_feature, split_criterion):
    """
    This function calculates information gain for splitting on 
    a particular descriptive feature for a given dataset
    and a given impurity criteria.
    Supported split criterion: 'entropy', 'gini'
    """
    
    print('target feature:', target)
    print('descriptive_feature:', descriptive_feature)
    print('split criterion:', split_criterion)
            
    target_entropy = compute_impurity(df[target], split_criterion)

    # we define two lists below:
    # entropy_list to store the entropy of each partition
    # weight_list to store the relative number of observations in each partition
    entropy_list = list()
    weight_list = list()
    
    # loop over each level of the descriptive feature
    # to partition the dataset with respect to that level
    # and compute the entropy and the weight of the level's partition
    for level in df[descriptive_feature].unique():
        df_feature_level = df[df[descriptive_feature] == level]
        entropy_level = compute_impurity(df_feature_level[target], split_criterion)
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level) / len(df)
        weight_list.append(round(weight_level, 3))

    print('impurity of partitions:', entropy_list)
    print('weights of partitions:', weight_list)

    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
    print('remaining impurity:', feature_remaining_impurity)
    
    information_gain = target_entropy - feature_remaining_impurity
    print('information gain:', information_gain)
    
    print('====================')

    return(information_gain, feature_remaining_impurity)

We now have to find Age thresholds, to split our dataframe in.

In [7]:
thresholds = list()

In [8]:
for i in range(1, df['Age'].count()):
    if df.iloc[i][TARGET] != df.iloc[i-1][TARGET]:
        
        # If the adjacent TARGET instances have different classifications, 
        # we store the mean age for this threshold in a list.
        
        t = ((df.iloc[i]['Age']+df.iloc[i-1]['Age'])/2)
        print("Threshold: >=", t)
        thresholds.append(t)

Threshold: >= 24.0
Threshold: >= 27.0
Threshold: >= 38.0
Threshold: >= 42.0


Above are the possible threshold points for Age feature, where we can possibly split our dataframe.<br><br>
Now we define our new **df_splits** dataframe

In [9]:
df_splits = pd.DataFrame(columns=['Split', 'Remainder', 'Information_Gain', 'Is_Optimal'])

We now calculate **Information Gain** of each subset, for all splits at each threshold<br>
We then add a 'Age_T' column to dataframe which has boolean values at threshold 'T'. For example, if threshold T >= 24, then column 'Age_24' will be False for all ages < 24 and True for all ages >= 24<br>
We then append all the data to df_splits dataframe

In [10]:
split_criterion = 'gini'
for t in thresholds:
    df_new = df.copy()
    col_name = "Age_" + str(int(t))
    df_new[col_name] = df_new[['Age']] >= t
    print("THRESHOLD: >=", t)
    feature_info_gain, remaining_impurity = comp_feature_information_gain(df_new, TARGET, col_name, split_criterion)
    df_splits = df_splits.append({'Split':col_name, 'Remainder':remaining_impurity, 'Information_Gain':feature_info_gain}, ignore_index=True)

THRESHOLD: >= 24.0
target feature: Annual_Income
descriptive_feature: Age_24
split criterion: gini
impurity of partitions: [0.0, 0.415]
weights of partitions: [0.15, 0.85]
remaining impurity: 0.35274999999999995
information gain: 0.2022500000000001
THRESHOLD: >= 27.0
target feature: Annual_Income
descriptive_feature: Age_27
split criterion: gini
impurity of partitions: [0.48, 0.32]
weights of partitions: [0.25, 0.75]
remaining impurity: 0.36
information gain: 0.19500000000000006
THRESHOLD: >= 38.0
target feature: Annual_Income
descriptive_feature: Age_38
split criterion: gini
impurity of partitions: [0.569, 0.469]
weights of partitions: [0.6, 0.4]
remaining impurity: 0.5289999999999999
information gain: 0.026000000000000134
THRESHOLD: >= 42.0
target feature: Annual_Income
descriptive_feature: Age_42
split criterion: gini
impurity of partitions: [0.631, 0.0]
weights of partitions: [0.75, 0.25]
remaining impurity: 0.47325
information gain: 0.08175000000000004


ID Feature is redundant, and hence is dropped.<br>
We have translated Age feature to different columns by thresholds, and added it to **df_splits** and hence Age column can be dropped.

In [11]:
df_new = df.copy()
df_new = df_new.drop(columns = ['ID', 'Age'])

In [12]:
df_new.head()

,Education,Marital_Status,Occupation,Annual_Income
2,high school,never married,agriculture,low
5,high school,never married,agriculture,low
12,bachelors,never married,agriculture,low
19,bachelors,married,transport,high
13,high school,married,professional,high


In [13]:
split_criterion = 'gini'
for feature in df_new.drop(columns='Annual_Income').columns:
    feature_info_gain, remaining_impurity = comp_feature_information_gain(df_new, 'Annual_Income', feature, split_criterion)
    df_splits = df_splits.append({'Split':feature, 'Remainder':remaining_impurity, 'Information_Gain':feature_info_gain}, ignore_index=True)

target feature: Annual_Income
descriptive_feature: Education
split criterion: gini
impurity of partitions: [0.625, 0.531, 0.375]
weights of partitions: [0.4, 0.4, 0.2]
remaining impurity: 0.5374000000000001
information gain: 0.01759999999999995
target feature: Annual_Income
descriptive_feature: Marital_Status
split criterion: gini
impurity of partitions: [0.611, 0.42, 0.375]
weights of partitions: [0.3, 0.5, 0.2]
remaining impurity: 0.4683
information gain: 0.08670000000000005
target feature: Annual_Income
descriptive_feature: Occupation
split criterion: gini
impurity of partitions: [0.5, 0.278, 0.5]
weights of partitions: [0.3, 0.3, 0.4]
remaining impurity: 0.4334
information gain: 0.12160000000000004


We set **Is_Optimal** feature to **True** for max Information_Gain row, all other to **False**

In [14]:
df_splits.loc[df_splits['Information_Gain'].idxmax(), 'Is_Optimal'] = True
df_splits['Is_Optimal'].fillna(False, inplace=True)

In [15]:
df_splits

,Split,Remainder,Information_Gain,Is_Optimal
0,Age_24,0.35275,0.20225,True
1,Age_27,0.36000,0.19500,False
2,Age_38,0.52900,0.02600,False
3,Age_42,0.47325,0.08175,False
4,Education,0.53740,0.01760,False
5,Marital_Status,0.46830,0.08670,False
6,Occupation,0.43340,0.12160,False


The above table shows final df_splits dataframe, with optimal root node **'Age_24'** marked as **True**.

## Part C
In this part we assume **Education** feature is the root node of decision tree.<br>
Now, we drop redundant columns ID and Age

In [16]:
df_new = df.copy().drop(columns=['ID', 'Age'])

In [17]:
df_new.head()

,Education,Marital_Status,Occupation,Annual_Income
2,high school,never married,agriculture,low
5,high school,never married,agriculture,low
12,bachelors,never married,agriculture,low
19,bachelors,married,transport,high
13,high school,married,professional,high


In [18]:
df_new.shape

(20, 4)

We list all unique values for column **Education** and look for possible missing values.

In [19]:
df_new['Education'].value_counts()

high school    8
bachelors      8
doctorate      4
Name: Education, dtype: int64

We create a dataframe called **df_prediction** to store our probabilities, at different leaf conditions.

In [20]:
df_prediction = pd.DataFrame(columns=['Leaf_Condition','Low_Income_Prob','Mid_Income_Prob','High_Income_Prob','Leaf_Prediction'])
df_prediction 

,Leaf_Condition,Low_Income_Prob,Mid_Income_Prob,High_Income_Prob,Leaf_Prediction


We write a function to calculate probabilities of low, mid and high income based on input df

In [21]:
def probs(df, df_orig):
    """This function returns low, mid, high probabilities of input df
    """
    low = mid = high = 0
    
    # probability = no. of instances of target class in this subset / no. of instances of target class in original dataset.
    
    if 'low' in df['Annual_Income'].values:
        low = df['Annual_Income'].value_counts()['low'] / df_orig['Annual_Income'].value_counts()['low'].sum()
    if 'mid' in df['Annual_Income'].values:
        mid = df['Annual_Income'].value_counts()['mid'] / df_orig['Annual_Income'].value_counts()['mid'].sum()
    if 'high' in df['Annual_Income'].values:
        high = df['Annual_Income'].value_counts()['high'] / df_orig['Annual_Income'].value_counts()['high'].sum()
        
    return low, mid, high

We now calculate **Information Gain** for all unique levels of **Education**

In [22]:
split_criterion = 'gini'
for condition in df_new['Education'].unique():
    df_temp = df_new[df_new.Education == condition]
    print(df_temp)
    low, mid, high = probs(df_temp, df_new)
    print("low:", low, "mid:", mid, "high:", high)
    print("=====================")
    condition_str = "Education == " + condition
    df_prediction = df_prediction.append({'Leaf_Condition':condition_str, 'Low_Income_Prob':low, 'Mid_Income_Prob':mid, 'High_Income_Prob':high},ignore_index=True)
    for col in df_temp.drop(columns=['Education', TARGET]):
        comp_feature_information_gain(df_temp, TARGET, col, split_criterion)
    

      Education Marital_Status    Occupation Annual_Income
2   high school  never married   agriculture           low
5   high school  never married   agriculture           low
13  high school        married  professional          high
9   high school        married     transport           mid
10  high school  never married     transport           mid
4   high school        married   agriculture           mid
18  high school       divorced  professional          high
6   high school       divorced     transport           mid
low: 0.6666666666666666 mid: 0.3333333333333333 high: 0.4
target feature: Annual_Income
descriptive_feature: Marital_Status
split criterion: gini
impurity of partitions: [0.444, 0.444, 0.5]
weights of partitions: [0.375, 0.375, 0.25]
remaining impurity: 0.458
information gain: 0.16699999999999998
target feature: Annual_Income
descriptive_feature: Occupation
split criterion: gini
impurity of partitions: [0.444, 0.0, 0.0]
weights of partitions: [0.375, 0.25, 0.375]
r

This function returns target class based on column name for highest vale.

In [23]:
def ret_target_class(str):
    classes = {'Mid_Income_Prob':'mid', 'Low_Income_Prob':'low', 'High_Income_Prob':'high'}
    return classes.get(str, None)

In [24]:
df_prediction['Leaf_Prediction'][0] = ret_target_class(df_prediction.drop(columns=['Leaf_Condition', 'Leaf_Prediction']).iloc[0].idxmax())
df_prediction['Leaf_Prediction'][1] = ret_target_class(df_prediction.drop(columns=['Leaf_Condition', 'Leaf_Prediction']).iloc[1].idxmax())
df_prediction['Leaf_Prediction'][2] = ret_target_class(df_prediction.drop(columns=['Leaf_Condition', 'Leaf_Prediction']).iloc[2].idxmax())

## Question 1 wrap up

In [25]:
df_splits

,Split,Remainder,Information_Gain,Is_Optimal
0,Age_24,0.35275,0.20225,True
1,Age_27,0.36000,0.19500,False
2,Age_38,0.52900,0.02600,False
3,Age_42,0.47325,0.08175,False
4,Education,0.53740,0.01760,False
5,Marital_Status,0.46830,0.08670,False
6,Occupation,0.43340,0.12160,False


In [26]:
df_prediction

,Leaf_Condition,Low_Income_Prob,Mid_Income_Prob,High_Income_Prob,Leaf_Prediction
0,Education == high school,0.666667,0.333333,0.4,low
1,Education == bachelors,0.333333,0.416667,0.4,mid
2,Education == doctorate,0.000000,0.250000,0.2,mid
